In [551]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import warnings

warnings.filterwarnings('ignore')



In [552]:

df = pd.read_csv("Methane_final.csv")
df.drop(columns=['Unnamed: 0'],inplace=True)
df = df[(df['country'] != 'World')  ]
df


,region,country,emissions,type,segment,reason,baseYear,notes
0,Africa,Algeria,257.611206,Agriculture,Total,All,2019-2021,Average based on United Nations Framework Conv...
1,Africa,Algeria,0.052000,Energy,Bioenergy,All,2022,Estimates from end-uses are for 2020 or 2021 (...
2,Africa,Algeria,130.798996,Energy,Gas pipelines and LNG facilities,Fugitive,2022,Not available
3,Africa,Algeria,69.741898,Energy,Gas pipelines and LNG facilities,Vented,2022,Not available
4,Africa,Algeria,213.987000,Energy,Onshore gas,Fugitive,2022,Not available
...,...,...,...,...,...,...,...,...
1521,Russia & Caspian,Uzbekistan,16.973917,Energy,Other from oil and gas,All,2022,Estimates from end-uses are for 2020 or 2021 (...
1522,Russia & Caspian,Uzbekistan,18.299999,Energy,Satellite-detected large oil and gas emissions,All,2022,Not available
1523,Russia & Caspian,Uzbekistan,780.916138,Energy,Total,All,2022,Estimates from end-uses are for 2020 or 2021 (...
1524,Russia & Caspian,Uzbekistan,3.845616,Other,Total,All,2019-2021,Average based on United Nations Framework Conv...


In [553]:
connection = sqlite3.connect("methane.db")


In [554]:
cursor= df.to_sql(name="methane",
          con=connection,
          if_exists="replace",
          index=False,
          dtype={'region':'TEXT',
                'country':'TEXT',
                'emissions':'REAL',
                'type':'TEXT',
                'segment':'TEXT',
                'reason':'TEXT',
                'baseYear':'TEXT',
                'notes':'TEXT'})

cursor = connection.cursor()

In [555]:
emissions = cursor.execute("SELECT region as Region, SUM(emissions) as Emissions FROM methane WHERE region IS NOT 'World' GROUP BY region ORDER BY Emissions desc ")
emission=pd.DataFrame(emissions, columns=["Region", "Emissions"])
print(emission)


                      Region      Emissions
0               Asia Pacific  177699.611465
1              North America   63347.409177
2           Russia & Caspian   62421.612360
3                     Africa   54197.152613
4                     Europe   49208.996818
5  Central and South America   45639.989799
6                Middle East   38095.798763
7                      Other   15997.357175


In [556]:
fig = px.bar(emission, y='Emissions', x='Region', text_auto='.2s',
            title="Total Methane Emmissions by Region",color='Region')
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.update_layout(legend=dict(title='Regions', orientation='v', yanchor='top', y=1, xanchor='left', x=1))
# Resize the figure
fig.update_layout(height=600, width=800)
# Center the title
fig.update_layout(title_x=0.5)
fig.show()

In [557]:
fig = px.pie(emission, values='Emissions', names='Region',)
# Add title
fig.update_layout(title='Emissions in Different Regions', title_font_size=24, title_font_family='Arial',title_x=0.5)

# Add legend and adjust position
fig.update_layout(legend=dict(title='Countries', orientation='v', yanchor='top', y=0.8, xanchor='left', x=1))
fig.update_layout(height=600, width=800)
# Center the title
fig.update_layout(title_x=0.5)
fig.show()


In [587]:
query = "SELECT region as Region, country as Country, SUM(emissions) as Emissions FROM methane WHERE country IS NOT 'World' AND country IS NOT 'Other' AND country IS NOT 'Other countries in Southeast Asia' AND COUNTRY IS NOT 'Other EU17 countries' AND COUNTRY IS NOT 'European Union' AND COUNTRY IS NOT 'Other countries in Europe' GROUP BY country, region ORDER BY SUM(emissions) desc LIMIT 10"
cursor.execute(query)
result=pd.DataFrame(cursor.fetchall(),columns=['Region', 'Country', 'Emissions'])

result

,Region,Country,Emissions
0,Asia Pacific,China,81048.371586
1,North America,United States,48604.877296
2,Russia & Caspian,Russia,42432.929804
3,Asia Pacific,India,34852.007386
4,Central and South America,Brazil,21720.838126
5,Asia Pacific,Indonesia,19404.469000
6,Middle East,Iran,13030.685366
7,Africa,Nigeria,9903.895667
8,Russia & Caspian,Turkmenistan,9834.398293
9,Asia Pacific,Pakistan,8170.627576


In [559]:
fig = px.bar(result, y='Emissions', x='Country', text_auto='.2s',color='Country'
            )
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.update_layout(title="Top 10 Countries with The Most Emission" , title_font_size=24, title_font_family='Arial',title_x=0.5)

fig.update_layout(height=600, width=800)
# Center the title
fig.update_layout(title_x=0.5)
fig.update_layout(showlegend=False)
fig.show()


In [560]:


fig = go.Figure(data=[go.Pie(labels=result['Country'], values=result['Emissions'])])
fig.update_traces(hoverinfo='percent', textinfo='percent', textfont_size=14, )

# Add title
fig.update_layout(title="Top 10 Countries with The Most Emission", title_font_size=24, title_font_family='Arial',title_x=0.5)

# Add legend and adjust position
fig.update_layout(legend=dict(title='Countries', orientation='v', yanchor='top', y=0.8, xanchor='left', x=1))

# Resize the figure
fig.update_layout(height=600, width=800)

# Show the chart
fig.show()


In [561]:
df.describe()

,emissions
count,1526.000000
mean,331.984226
std,1451.481353
min,0.000459
25%,2.588755
50%,22.501150
75%,114.979780
max,25372.222656


In [562]:
query = "SELECT segment as Segment, SUM(emissions) as Emissions FROM methane WHERE region IS NOT 'World' AND segment IS NOT 'Total' GROUP BY segment ORDER BY Emissions desc"
cursor.execute(query)
segment=pd.DataFrame(cursor.fetchall(),columns=['Segment', 'Emissions'])
segment

,Segment,Emissions
0,Onshore oil,34332.783240
1,Steam coal,30047.273344
2,Onshore gas,18097.597573
3,Gas pipelines and LNG facilities,12639.023727
4,Coking coal,10313.157684
5,Bioenergy,9926.408038
6,Offshore oil,8349.611860
7,Offshore gas,4342.396777
8,Satellite-detected large oil and gas emissions,3106.400007
9,Other from oil and gas,2493.177699


In [563]:
fig = px.bar(segment, x='Segment', y='Emissions', color='Segment',
              opacity=0.8)
fig.update_layout(title='Emissions divided by segments', xaxis_title='Segment',
                  yaxis_title='Emissions', xaxis_tickangle=0,title_font_size=24)
fig.update_xaxes(showticklabels=False)

fig.update_layout(height=600, width=1080)
# Center the title
fig.update_layout(title_x=0.5)
fig.show()

In [564]:

fig = go.Figure(data=[go.Pie(labels=segment['Segment'], values=segment['Emissions'])])
fig.update_traces(hoverinfo='percent', textinfo='percent', textfont_size=14, )

# Add title
fig.update_layout(title='Emissions divided by segments', title_font_size=24, title_font_family='Arial',title_x=0.5)

# Add legend and adjust position
fig.update_layout(legend=dict(title='Countries', orientation='v', yanchor='top', y=0.8, xanchor='left', x=1))

# Resize the figure
fig.update_layout(height=600, width=800)

# Show the chart
fig.show()

In [565]:
df

,region,country,emissions,type,segment,reason,baseYear,notes
0,Africa,Algeria,257.611206,Agriculture,Total,All,2019-2021,Average based on United Nations Framework Conv...
1,Africa,Algeria,0.052000,Energy,Bioenergy,All,2022,Estimates from end-uses are for 2020 or 2021 (...
2,Africa,Algeria,130.798996,Energy,Gas pipelines and LNG facilities,Fugitive,2022,Not available
3,Africa,Algeria,69.741898,Energy,Gas pipelines and LNG facilities,Vented,2022,Not available
4,Africa,Algeria,213.987000,Energy,Onshore gas,Fugitive,2022,Not available
...,...,...,...,...,...,...,...,...
1521,Russia & Caspian,Uzbekistan,16.973917,Energy,Other from oil and gas,All,2022,Estimates from end-uses are for 2020 or 2021 (...
1522,Russia & Caspian,Uzbekistan,18.299999,Energy,Satellite-detected large oil and gas emissions,All,2022,Not available
1523,Russia & Caspian,Uzbekistan,780.916138,Energy,Total,All,2022,Estimates from end-uses are for 2020 or 2021 (...
1524,Russia & Caspian,Uzbekistan,3.845616,Other,Total,All,2019-2021,Average based on United Nations Framework Conv...


In [566]:
query3 = "SELECT type, SUM(emissions) as Emissions FROM methane WHERE region IS NOT 'World'  GROUP BY type ORDER BY Emissions desc"
cursor.execute(query3)
types=pd.DataFrame(cursor.fetchall(),columns=['Type', 'Emissions'])
types

,Type,Emissions
0,Energy,270694.345156
1,Agriculture,150805.103144
2,Waste,75079.707008
3,Other,10028.772862


In [567]:
fig = px.pie(types, values='Emissions', names='Type',)
# Add title
fig.update_layout(title='Emissions Devided by Sector', title_font_size=24, title_font_family='Arial',title_x=0.5)
fig.update_traces(hoverinfo='percent', textinfo='percent', textfont_size=22, )
# Add legend and adjust position
fig.update_layout(legend=dict(title='Sector', orientation='v', yanchor='top', y=0.8, xanchor='left', x=1))
fig.update_layout(height=600, width=800)
# Center the title
fig.update_layout(title_x=0.5)
fig.show()

In [568]:
connection.commit()


In [586]:
import plotly.express as px

# Filter out rows where region is 'World' or segment is 'Total' or type is not 'Energy'
energy_data = df[(df['region'] != 'World') & (df['segment'] != 'Total') & (df['type'] == 'Energy')]

# Create sunburst plot for Energy emissions
fig = px.sunburst(energy_data, path=['region', 'segment'], values='emissions', color='emissions',
                  color_continuous_scale='oranges', title='Region-wise Segment Analysis of Energy Emissions',
                  height=600, width=600)
fig.update_layout(title_x=0.5)
# Show the plot
fig.show()

In [570]:
query = "SELECT region as Region, country as Country, SUM(emissions) as Emissions FROM methane WHERE country IS NOT 'World' AND country IS NOT 'Other' AND country IS NOT 'Other countries in Southeast Asia' AND COUNTRY IS NOT 'Other EU17 countries' AND COUNTRY IS NOT 'European Union' AND COUNTRY IS NOT 'Other countries in Europe' GROUP BY country, region ORDER BY SUM(emissions) desc "
cursor.execute(query)
emico=pd.DataFrame(cursor.fetchall(),columns=['Region', 'Country', 'Emissions'])
emico

,Region,Country,Emissions
0,Asia Pacific,China,81048.371586
1,North America,United States,48604.877296
2,Russia & Caspian,Russia,42432.929804
3,Asia Pacific,India,34852.007386
4,Central and South America,Brazil,21720.838126
...,...,...,...
94,Africa,Guinea-Bissau,87.772880
95,Europe,Slovenia,83.387466
96,Africa,Gambia,47.587691
97,Africa,Liberia,43.897755


In [571]:


# create the Choropleth Map
fig = px.choropleth(emico, locations="Country", 
                    locationmode='country names', 
                    color="Emissions",
                    hover_name="Country", 
                    color_continuous_scale='oranges',
                    )
fig.update_layout(title="Emissions Map")
fig.update_layout(height=600, width=1080)
# Center the title
fig.update_layout(title_x=0.5)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [572]:
worldpop=pd.read_csv('world_population.csv')
worldpop=worldpop.rename(columns={'Country/Territory': 'Country'})

In [573]:
merged = pd.merge(worldpop, emico, on='Country')

# print the merged table
merged

,Rank,CCA3,Country,Capital,Continent,2022 Population,2020 Population,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage,Region,Emissions
0,34,DZA,Algeria,Algiers,Africa,44903225,43451666,39543154,35856344,30774621,25518074,18739378,13795915,2381741,18.8531,1.0164,0.56,Africa,6113.441756
1,42,AGO,Angola,Luanda,Africa,35588987,33428485,28127721,23364185,16394062,11828638,8330047,6029700,1246700,28.5466,1.0315,0.45,Africa,1771.640266
2,33,ARG,Argentina,Buenos Aires,South America,45510318,45036032,43257065,41100123,37070774,32637657,28024803,23842803,2780400,16.3683,1.0052,0.57,Central and South America,6558.256740
3,55,AUS,Australia,Canberra,Oceania,26177413,25670051,23820236,22019168,19017963,17048003,14706322,12595034,7692024,3.4032,1.0099,0.33,Asia Pacific,7775.794053
4,91,AZE,Azerbaijan,Baku,Asia,10358074,10284951,9863480,9237202,8190337,7427836,6383060,5425317,86600,119.6082,1.0044,0.13,Russia & Caspian,946.942928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,133,URY,Uruguay,Montevideo,South America,3422794,3429086,3402818,3352651,3292224,3117012,2953750,2790265,181034,18.9069,0.9990,0.04,Central and South America,908.923504
90,43,UZB,Uzbekistan,Tashkent,Asia,34627652,33526656,30949417,28614227,24925554,20579100,15947129,12011361,447400,77.3975,1.0160,0.43,Russia & Caspian,2747.778754
91,51,VEN,Venezuela,Caracas,South America,28301696,28490453,30529716,28715022,24427729,19750579,15210443,11355475,916445,30.8820,1.0036,0.35,Central and South America,6691.244875
92,16,VNM,Vietnam,Hanoi,Asia,98186856,96648685,92191398,87411012,79001142,66912613,52968270,41928849,331212,296.4472,1.0074,1.23,Asia Pacific,3133.670373


In [575]:
query4 = "SELECT type, SUM(emissions) as Emissions FROM methane WHERE region IS NOT 'World' AND type IS NOT 'Total' GROUP BY type ORDER BY Emissions desc"
cursor.execute(query4)
types=pd.DataFrame(cursor.fetchall(),columns=['Type', 'Emissions'])
types

,Type,Emissions
0,Energy,270694.345156
1,Agriculture,150805.103144
2,Waste,75079.707008
3,Other,10028.772862


In [576]:
fig = px.sunburst(df, 
                  path=["country", "type",], 
                  values="emissions",
                  color="emissions"
                  
                  )

# Set the title
fig.update_layout(title="Emissions in Each Country Divided by Sector Hierarchy")

fig.update_layout(height=600, width=800)
# Center the title
fig.update_layout(title_x=0.5)
# Show the chart
fig.show()

In [577]:
# Create a scatter plot
fig = px.scatter(merged, x="2022 Population", y="Emissions", trendline="ols", text="Country")

# Update the layout to adjust font size and add title
fig.update_layout(title="The Correlation Between Emissions and 2022 Population",
                  title_x=0.5, 
                  xaxis_title="2022 Population", 
                  yaxis_title="Emissions", 
                  font=dict(size=12))

# Add annotations for the text labels
fig.update_traces(textposition="top center")
fig.update_layout(height=600, width=1080)
# Center the title
fig.update_layout(title_x=0.5)
fig.show()

In [582]:
df

,region,country,emissions,type,segment,reason,baseYear,notes
0,Africa,Algeria,257.611206,Agriculture,Total,All,2019-2021,Average based on United Nations Framework Conv...
1,Africa,Algeria,0.052000,Energy,Bioenergy,All,2022,Estimates from end-uses are for 2020 or 2021 (...
2,Africa,Algeria,130.798996,Energy,Gas pipelines and LNG facilities,Fugitive,2022,Not available
3,Africa,Algeria,69.741898,Energy,Gas pipelines and LNG facilities,Vented,2022,Not available
4,Africa,Algeria,213.987000,Energy,Onshore gas,Fugitive,2022,Not available
...,...,...,...,...,...,...,...,...
1521,Russia & Caspian,Uzbekistan,16.973917,Energy,Other from oil and gas,All,2022,Estimates from end-uses are for 2020 or 2021 (...
1522,Russia & Caspian,Uzbekistan,18.299999,Energy,Satellite-detected large oil and gas emissions,All,2022,Not available
1523,Russia & Caspian,Uzbekistan,780.916138,Energy,Total,All,2022,Estimates from end-uses are for 2020 or 2021 (...
1524,Russia & Caspian,Uzbekistan,3.845616,Other,Total,All,2019-2021,Average based on United Nations Framework Conv...


In [583]:
region_pivot = df.pivot_table(values='emissions', index='region', columns='type', aggfunc='sum')
fig = px.imshow(region_pivot, x=region_pivot.columns, y=region_pivot.index, 
                labels=dict(x="Sector", y="Region", z="Emissions"),
                color_continuous_scale='YlOrRd',
                text_auto=True)
fig.update_layout(title="Heatmap of Emissions by Sector and Region",
                  xaxis_nticks=len(region_pivot.columns),
                  yaxis_nticks=len(region_pivot.index))

fig.update_layout(height=800, width=600)
# Center the title
fig.update_layout(title_x=0.5)

fig.show()

In [584]:
# create a treemap chart
fig = px.treemap(df, 
                 path=['country', 'type'], 
                 values='emissions',
                 color='emissions',
                 color_continuous_scale='YlOrRd')

# customize the layout
fig.update_layout(
    title='Proportion of Emissions by Country and Sector',
    font=dict(size=16),
    margin=dict(l=10, r=10, t=60, b=10)
)
fig.update_layout(title_x=0.5)
# show the chart
fig.show()